In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
BATCH=20

In [7]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data/play/mnist', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
])),
     batch_size=BATCH, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data/play/mnist', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
batch_size=BATCH, shuffle=True)

In [11]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
100. * correct / len(test_loader.dataset)))

In [18]:
model = Net()
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1, 3):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319453
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.319855
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.276029
Train Epoch: 1 [1200/60000 (2%)]	Loss: 2.301801
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.295629
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.234958
Train Epoch: 1 [2400/60000 (4%)]	Loss: 2.275779
Train Epoch: 1 [2800/60000 (5%)]	Loss: 2.233505
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.247890
Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.244702
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.241807
Train Epoch: 1 [4400/60000 (7%)]	Loss: 2.226342
